In [1]:
%reload_ext autoreload
%autoreload 2

In [12]:
import pandas as pd 
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
import requests
import ast
from glob import glob
import os

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [6]:
df = pd.read_csv('data/0.fighters_raw.csv', index_col=0)
df['photos'] = df['photos'].replace({'[]':np.nan})

df

,armSpan,avgFightTime,city,country,dateOfBirth,disciplines,draws,height,id,knockdownsPerFight,...,significantStrikesMissedPerMinute,submissionAttemptsPer15minutes,takedownsAccuracy,takedownsDefendPercent,takedownsPer15min,timezone,weight,weightCategory.id,weightCategory.name,wins
0,190.50,NaN,Bonnyville,Canada,1991-08-02,[],1,187.96,1,0.00,...,3.23,NaN,NaN,100.0,NaN,America/Edmonton,115.67,9,Тяжелый вес,17
1,190.50,NaN,NaN,Brazil,1989-06-23,[],0,190.50,2,0.00,...,NaN,NaN,NaN,NaN,NaN,America/Sao_Paulo,112.04,9,Тяжелый вес,6
2,193.04,11:41,Makhachkala,Dagestan,1981-09-02,[],0,190.50,3,0.21,...,2.45,0.2,24.0,47.0,1.23,Europe/Moscow,106.59,9,Тяжелый вес,20
3,187.96,15:00,Manaus,Brazil,1992-12-24,[],0,182.88,4,0.00,...,2.90,NaN,20.0,80.0,0.64,America/Manaus,92.99,8,Полутяжелый вес,15
4,190.50,13:05,Ikuno,Japan,1975-07-29,[],0,177.80,5,0.00,...,3.62,1.0,67.0,92.0,2.29,Asia/Tokyo,77.11,6,Полусредний вес,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3664,NaN,NaN,NaN,NaN,1991-09-28,[],0,182.88,3673,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,83.91,14,Не определена,18
3665,NaN,NaN,NaN,NaN,1987-07-09,[],0,182.88,3674,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,77.11,14,Не определена,15
3666,NaN,NaN,NaN,NaN,1994-08-19,[],0,170.18,3675,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,65.77,14,Не определена,11
3667,NaN,NaN,NaN,NaN,1990-11-09,[],0,175.26,3676,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,65.77,14,Не определена,13


In [9]:
photos = df.loc[0, 'photos']
photos = ast.literal_eval(photos)
photos

[{'url': 'http://bet2face.com/uploads/images/fighters/tannerboser-5d3570e3c8d49.png',
  'isFullFace': True,
  'isProfile': False},
 {'url': 'http://bet2face.com/uploads/images/fighters/tannerboser-5d2c38e596414.jpeg',
  'isFullFace': False,
  'isProfile': True},
 {'url': 'http://bet2face.com/uploads/images/fighters/tannerboser-5d35711a963ed.png',
  'isFullFace': False,
  'isProfile': False},
 {'url': 'http://bet2face.com/uploads/images/fighters/tannerboser-5d3570e3ceca4.png',
  'isFullFace': False,
  'isProfile': False},
 {'url': 'http://bet2face.com/uploads/images/fighters/tannerboser-5d3570e3cc279.png',
  'isFullFace': False,
  'isProfile': False},
 {'url': 'http://bet2face.com/uploads/images/fighters/tannerboser-5d3570d8ea36f.png',
  'isFullFace': False,
  'isProfile': False},
 {'url': 'http://bet2face.com/uploads/images/fighters/tannerboser-5d2c38e5908e5.jpeg',
  'isFullFace': False,
  'isProfile': False}]

In [16]:
for i in tqdm(df.index[:]):
    try:
        id_ = df.loc[i, 'id']
        photos = df.loc[i, 'photos']

        if photos == np.nan:
            continue

        photos = ast.literal_eval(photos)
        for photo in photos:
            url = photo['url']
            ending = url.split('.')[-1]

            if photo['isFullFace'] == True: 
                photo_path = 'data/photos/' + '{}_anfas.{}'.format(id_, ending)
                df.loc[i, 'anfas_photo'] = photo_path

            elif photo['isProfile'] == True: 
                photo_path = 'data/photos/' + '{}_profile.{}'.format(id_, ending)
                df.loc[i, 'profile_photo'] = photo_path

            with open(photo_path, 'wb') as handle:
                response = requests.get(url, stream=True, verify=False)

                if not response.ok:
                    print(response)

                for block in response.iter_content(1024):
                    if not block:
                        break
                    handle.write(block)
    except Exception as exp:
        print(i, exp)

/home/nikolay/anaconda3/envs/merlin/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


<Response [404]>
771 malformed node or string: nan
772 malformed node or string: nan
775 malformed node or string: nan
776 malformed node or string: nan
777 malformed node or string: nan
778 malformed node or string: nan
780 malformed node or string: nan
795 malformed node or string: nan
805 malformed node or string: nan
815 malformed node or string: nan
825 malformed node or string: nan
841 malformed node or string: nan
1430 malformed node or string: nan
1441 malformed node or string: nan
1442 malformed node or string: nan
1443 malformed node or string: nan
1446 malformed node or string: nan
1447 malformed node or string: nan
1449 malformed node or string: nan
1450 malformed node or string: nan
1456 malformed node or string: nan
1459 malformed node or string: nan
1464 malformed node or string: nan
1466 malformed node or string: nan
1467 malformed node or string: nan
1468 malformed node or string: nan
1469 malformed node or string: nan
1476 malformed node or string: nan
1477 malformed 

1727 malformed node or string: nan
1728 malformed node or string: nan
1729 malformed node or string: nan
1730 malformed node or string: nan
1731 malformed node or string: nan
1732 malformed node or string: nan
1733 malformed node or string: nan
1734 malformed node or string: nan
1735 malformed node or string: nan
1736 malformed node or string: nan
1737 malformed node or string: nan
1738 malformed node or string: nan
1739 malformed node or string: nan
1740 malformed node or string: nan
1741 malformed node or string: nan
1742 malformed node or string: nan
1743 malformed node or string: nan
1744 malformed node or string: nan
1745 malformed node or string: nan
1746 malformed node or string: nan
1747 malformed node or string: nan
1748 malformed node or string: nan
1749 malformed node or string: nan
1750 malformed node or string: nan
1751 malformed node or string: nan
1752 malformed node or string: nan
1753 malformed node or string: nan
1754 malformed node or string: nan
1755 malformed node 

3045 malformed node or string: nan
3046 malformed node or string: nan
3047 malformed node or string: nan
3048 malformed node or string: nan
3049 malformed node or string: nan
3050 malformed node or string: nan
3051 malformed node or string: nan
3052 malformed node or string: nan
3053 malformed node or string: nan
3054 malformed node or string: nan
3055 malformed node or string: nan
3056 malformed node or string: nan
3057 malformed node or string: nan
3058 malformed node or string: nan
3059 malformed node or string: nan
3060 malformed node or string: nan
3061 malformed node or string: nan
3062 malformed node or string: nan
3063 malformed node or string: nan
3064 malformed node or string: nan
3065 malformed node or string: nan
3066 malformed node or string: nan
3067 malformed node or string: nan
3068 malformed node or string: nan
3069 malformed node or string: nan
3070 malformed node or string: nan
3071 malformed node or string: nan
3072 malformed node or string: nan
3073 malformed node 

3384 malformed node or string: nan
3385 malformed node or string: nan
3389 malformed node or string: nan
3390 malformed node or string: nan
3393 malformed node or string: nan
3394 malformed node or string: nan
<Response [404]>
<Response [404]>
<Response [404]>
<Response [404]>
<Response [404]>
<Response [404]>
<Response [404]>
<Response [404]>
<Response [404]>
<Response [404]>
3400 malformed node or string: nan
3401 malformed node or string: nan
3402 malformed node or string: nan
3404 malformed node or string: nan
3406 malformed node or string: nan
3407 malformed node or string: nan
3408 malformed node or string: nan
3409 malformed node or string: nan
3410 malformed node or string: nan
3411 malformed node or string: nan
3412 malformed node or string: nan
3413 malformed node or string: nan
3414 malformed node or string: nan
3415 malformed node or string: nan
3416 malformed node or string: nan
3417 malformed node or string: nan
3419 malformed node or string: nan
3420 malformed node or st

In [ ]:
def convert_to_jpg():
    for i in tqdm(glob('photos/*')):
        img = cv2.imread(i)
        os.remove(i)
        new_name = i.split('.')[0]+'.jpg'
        cv2.imwrite(new_name, img)

